In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
from matplotlib import pyplot as plt

# Load YOLO model
model = YOLO('yolov8n.pt')  # Use nano model for speed

# Load video
video_path = 'data/full_video/tennis_full_video.mp4'
cap = cv2.VideoCapture(video_path)

# Read first frame
ret, frame = cap.read()
if not ret:
    print("Error: Could not read video")
else:
    # Apply the same crop as court_roi
    cropped_frame = frame
    
    # Detect people in the cropped frame
    results = model(cropped_frame, classes=[0], verbose=False)  # class 0 is 'person'
    
    # Get detections and sort by confidence
    detections = results[0].boxes
    if len(detections) > 0:
        # Convert to numpy for easier handling
        boxes = detections.xyxy.cpu().numpy()
        confidences = detections.conf.cpu().numpy()
        
        # Sort by confidence and take top 2
        sorted_indices = np.argsort(confidences)[::-1]
        top_2_indices = sorted_indices[:min(2, len(sorted_indices))]
        
        # Draw bounding boxes on cropped frame
        result_frame = cropped_frame.copy()
        colors = [(255, 0, 0), (0, 255, 0)]  # Red and Green for 2 players
        
        for i, idx in enumerate(top_2_indices):
            x1, y1, x2, y2 = boxes[idx].astype(int)
            conf = confidences[idx]
            
            # Draw bounding box
            cv2.rectangle(result_frame, (x1, y1), (x2, y2), colors[i], 2)
            
            # Add label
            label = f'Player {i+1}: {conf:.2f}'
            cv2.putText(result_frame, label, (x1, y1 - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, colors[i], 2)
        
        # Display result
        plt.figure(figsize=(12, 6))
        plt.imshow(cv2.cvtColor(result_frame, cv2.COLOR_BGR2RGB))
        plt.title(f'Player Tracking - {len(top_2_indices)} players detected')
        plt.axis('off')
        plt.show()
        
        print(f"Detected {len(top_2_indices)} players")
        for i, idx in enumerate(top_2_indices):
            print(f"Player {i+1}: Confidence = {confidences[idx]:.2f}")
    else:
        print("No players detected in frame")
        plt.figure(figsize=(12, 6))
        plt.imshow(cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB))
        plt.title('No players detected')
        plt.axis('off')
        plt.show()

cap.release()

Error: Could not read video


[ WARN:0@140.261] global cap_gstreamer.cpp:1436 open OpenCV | GStreamer warning: Error opening bin: no element "data"
[ WARN:0@140.261] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [3]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLO model
model = YOLO('yolov8n.pt')

# Load video
cap = cv2.VideoCapture('../data/full_video/tennis_full_video.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Crop to court ROI
    cropped_frame = frame
    
    # Detect people
    results = model(cropped_frame, classes=[0], verbose=False)
    detections = results[0].boxes
    
    # Draw on frame
    if len(detections) > 0:
        boxes = detections.xyxy.cpu().numpy()
        confidences = detections.conf.cpu().numpy()
        
        # Get top 2 players by confidence
        sorted_indices = np.argsort(confidences)[::-1][:2]
        
        # Assign players based on vertical position
        # Player 2 (green) = top half (y < 100)
        # Player 1 (red) = bottom half (y >= 100)
        players = []
        for idx in sorted_indices:
            x1, y1, x2, y2 = boxes[idx].astype(int)
            center_y = (y1 + y2) / 2
            
            if center_y < 200:
                player_id = 2  # Top player
                color = (0, 255, 0)  # Green
            else:
                player_id = 1  # Bottom player
                color = (255, 0, 0)  # Red
            
            players.append((x1, y1, x2, y2, player_id, color))
        
        # Draw bounding boxes
        for x1, y1, x2, y2, player_id, color in players:
            cv2.rectangle(cropped_frame, (x1, y1), (x2, y2), color, 2)
            # centre of mass
            center_x = (x1 + x2) / 2
            center_y = (y1 + y2) / 2
            cv2.circle(cropped_frame, (int(center_x), int(center_y)), 5, color, -1)
            cv2.putText(cropped_frame, f'P{player_id}', (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
    # Display
    cv2.imshow('Player Tracking', cropped_frame)
    
    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()